**Analsis exploratorio de los datos clinicos**

In [1]:
import os
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
from styles import *
from PIL import Image

In [2]:
df = pd.read_csv('./data/animal_disease_prediction_cleaned.csv')

In [3]:
disease_count = df['Disease_Prediction'].value_counts().head(8)
fig1 = go.Figure([go.Bar(x=disease_count.index, y=disease_count.values, marker_color='teal')])
fig1.update_layout(
    title="Top 8 Enfermedades más Frecuentes",
    xaxis_title="Enfermedad",
    yaxis_title="Frecuencia"
)
fig1 = bar_style(fig1)
fig1.show()

In [4]:
fig2 = go.Figure([
    go.Scatter(
        x=df['Body_Temperature'],
        y=df['Heart_Rate'],
        mode='markers',
        marker=dict(color='teal', size=8),
        opacity=0.6
    )
])
fig2.update_layout(
    title="Relación entre Temperatura Corporal y Frecuencia Cardíaca",
    xaxis_title="Temperatura Corporal",
    yaxis_title="Frecuencia Cardíaca"
)
fig2 = scatter_style(fig2)
fig2.show()

In [5]:

fig3 = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Top 5 Síntomas - Categoría 1', 'Top 5 Síntomas - Categoría 2', 
                   'Top 5 Síntomas - Categoría 3', 'Top 5 Síntomas - Categoría 4')
)

for i, symptom_col in enumerate(['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4'], 1):
    # Contar frecuencia de cada síntoma y obtener top 5
    symptom_counts = df[symptom_col].value_counts().head(5)
    
    row = (i-1) // 2 + 1
    col = (i-1) % 2 + 1
    
    fig3.add_trace(
        go.Bar(
            x=symptom_counts.index,
            y=symptom_counts.values,
            marker_color='teal',
            showlegend=False,
        ),
        row=row, col=col
    )
    
    fig3.update_xaxes(title_text="Síntoma", row=row, col=col)
    fig3.update_yaxes(title_text="Frecuencia", row=row, col=col)

fig3.update_layout(
    height=900,
    width=1400,
    showlegend=False,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)'
)

# Aplicar estilo a cada subplot
for i in range(1, 3):
    for j in range(1, 3):
        fig3.update_xaxes(showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=i, col=j)
        fig3.update_yaxes(showgrid=True, gridcolor='lightgrey', gridwidth=0.5, row=i, col=j)

fig3.show()

In [6]:
animal_count = df['Animal_Type'].value_counts()

fig4 = go.Figure([go.Pie(
    labels=animal_count.index,
    values=animal_count.values,
    marker=dict(colors=px.colors.qualitative.Set3),
    textinfo='label+percent',
    hole=0.3,
    pull=[0.1 if i == 0 else 0 for i in range(len(animal_count))]
)])

fig4.update_layout(
    title="Distribución por Tipo de Animal",
    annotations=[dict(text=f'Total: {sum(animal_count.values)}', x=0.5, y=0.5, showarrow=False)]
)

fig4 = pie_style(fig4)
fig4.show()

In [7]:
gender_animal = pd.crosstab(df['Animal_Type'], df['Gender'])
fig5 = go.Figure([
    go.Bar(name='Macho', x=gender_animal.index, y=gender_animal['Male'], marker_color='teal'),
    go.Bar(name='Hembra', x=gender_animal.index, y=gender_animal['Female'], marker_color='lightseagreen')
])
fig5.update_layout(
    title="Distribución de Género por Tipo de Animal",
    xaxis_title="Tipo de Animal",
    yaxis_title="Cantidad",
    barmode='stack'
)
fig5 = stacked_bar_style(fig5)
fig5.show()

**Analisis Exploratorio de las imagenes**


In [8]:
data = './data/pet_disease_images_augmented'

In [9]:
class_counts = {}
for disease_class in os.listdir(data):
    class_path = os.path.join(data, disease_class)
    if os.path.isdir(class_path):
        n_images = len(os.listdir(class_path))
        class_counts[disease_class] = n_images

fig1 = go.Figure([go.Pie(
    labels=list(class_counts.keys()),
    values=list(class_counts.values()),
    marker=dict(colors=px.colors.qualitative.Set3),
    textinfo='label+percent',
    hole=0.3,
    pull=[0.1 if i == 0 else 0 for i in range(len(class_counts))]
)])
fig1.update_layout(
    title="Distribución de Imágenes por Clase",
    annotations=[dict(text=f'Total: {sum(class_counts.values())}', x=0.5, y=0.5, showarrow=False)]
)
fig1 = pie_style(fig1, height=800)
fig1.show()

In [10]:
from tkinter import W


def plot_sample_images():
    fig = make_subplots(rows=2, cols=2, subplot_titles=list(class_counts.keys())[:4])
    
    for idx, disease_class in enumerate(list(class_counts.keys())[:4], 1):
        class_path = os.path.join(data, disease_class)
        sample_image = os.listdir(class_path)[0]
        img_path = os.path.join(class_path, sample_image)
        
        img = np.array(Image.open(img_path))
        
        row = (idx-1) // 2 + 1
        col = (idx-1) % 2 + 1
        
        fig.add_trace(
            go.Image(z=img),
            row=row, col=col
        )
    
    fig.update_layout(
        title={"text": "Ejemplos de Imágenes por Clase", "x": 0.5, "xanchor": "center"},
        height=800,
        width=1000
    )
    fig.show()

plot_sample_images()

In [11]:
animal_types = {'Cat': 0, 'Dog': 0}
for disease_class in class_counts.keys():
    if 'Cat' in disease_class:
        animal_types['Cat'] += class_counts[disease_class]
    elif 'Dog' in disease_class:
        animal_types['Dog'] += class_counts[disease_class]

fig4 = go.Figure([go.Pie(
    labels=list(animal_types.keys()),
    values=list(animal_types.values()),
    hole=0.3
)])
fig4.update_layout(title="Distribución de Imágenes por Tipo de Animal")
fig4 = pie_style(fig4)
fig4.show()